In [1]:
import py3Dmol

def graficar(pdb_path):
    with open(pdb_path, 'r') as f:
        pdb_data = f.read()
    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_data, 'pdb')
    view.setStyle({'cartoon': {'color': 'spectrum'}})  # Ribbon-like style
    view.addSurface(py3Dmol.VDW, {'opacity': 0.2, 'color':'white'})
    view.zoomTo()
    view.show()

# Ejemplo de uso:
graficar(r"E.coli.pdb")

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [2]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from pdbfixer import PDBFixer
from sys import stdout

# === PARTE 1: Cargar el archivo PDB de insulina ===
input_pdb_path = "E.coli.pdb"  # Asegúrate de tener este archivo en el mismo directorio
fixer = PDBFixer(filename=input_pdb_path)

# Arreglar posibles errores y agregar agua
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(pH=7.0)

# === PARTE 2: Crear el sistema en OpenMM ===
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
modeller = Modeller(fixer.topology, fixer.positions)
modeller.addSolvent(forcefield, model='tip3p', padding=1.0*nanometer)

system = forcefield.createSystem(modeller.topology,
                                 nonbondedMethod=PME,
                                 nonbondedCutoff=1.0*nanometer,
                                 constraints=HBonds)

# === PARTE 3: Configurar la simulación con alta temperatura ===
integrator = LangevinIntegrator(1000*kelvin, 1/picosecond, 0.002*picoseconds)  # 500 K para desnaturalizar
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

# Minimizar energía
print("Minimizando energía...")
simulation.minimizeEnergy()

# Equilibrar durante un corto tiempo
print("Equilibrando...")
simulation.context.setVelocitiesToTemperature(1000*kelvin)
simulation.step(1000)

# Simular (por ejemplo 50 ps)
n_steps = 25000  # 50 ps con paso de 2 fs
print("Simulando desnaturalización...")
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, progress=True, remainingTime=True, speed=True, totalSteps=n_steps, separator='\t'))
simulation.step(n_steps)

# === PARTE 4: Guardar el estado final ===
positions = simulation.context.getState(getPositions=True).getPositions()
with open("E.coli_desnaturalizada_2.pdb", "w") as output:
    PDBFile.writeFile(simulation.topology, positions, output)

print("\n✅ Desnaturalización completada. Archivo guardado como 'E.coli_densnaturalizada_2.pdb'.")

Minimizando energía...
Equilibrando...
Simulando desnaturalización...
#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
8.0%	2000	-167516.3850087209	995.5574828158632	--	--
12.0%	3000	-168255.1115712209	1000.2692841531671	228	0:16
16.0%	4000	-168524.8244618459	1018.0576868487212	228	0:15
20.0%	5000	-166350.3371571584	998.9490630565679	225	0:15
24.0%	6000	-167204.9055165334	1003.8338329198391	224	0:14
28.0%	7000	-167546.2678212209	990.2385720464443	225	0:13
32.0%	8000	-169374.7658680959	1005.5643545907047	225	0:13
36.0%	9000	-168121.9250477834	995.8307910348074	226	0:12
40.0%	10000	-167953.0812977834	1005.9996259866389	226	0:11
44.0%	11000	-167606.0510243459	998.8201225111144	226	0:10
48.0%	12000	-166644.3508290334	1011.1902546082134	226	0:09
52.0%	13000	-168991.6457509084	1007.6698515592913	226	0:09
56.0%	14000	-168189.4416493459	1007.686155100785	226	0:08
60.0%	15000	-167716.7619618459	991.9152506827911	226	0:07
64.0%	16000	-166364

In [3]:
import py3Dmol

def graficar(pdb_path):
    with open(pdb_path, 'r') as f:
        pdb_data = f.read()
    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_data, 'pdb')
    view.setStyle({'cartoon': {'color': 'spectrum'}})  # Ribbon-like style
    view.addSurface(py3Dmol.VDW, {'opacity': 0.2, 'color':'white'})
    view.zoomTo()
    view.show()

# Ejemplo de uso:
graficar(r"E.coli_desnaturalizada_2.pdb")

3Dmol.js failed to load for some reason. Please check your browser console for error messages.